In [1]:
!pip install numpy matplotlib pillow torch torchvision


  Using cached matplotlib-3.10.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached torchvision-0.21.0-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
Using cached matplotlib-3.10.1-cp313-cp313-win_amd64.whl (8.1 MB)
Using cached torchvision-0.21.0-cp313-cp313-win_amd64.whl (1.6 MB)
Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl (220 kB)


In [ ]:
# Step 1: Import Required Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageFilter
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 2: Define Image Processing Techniques
def apply_image_processing(image, technique):
    if technique == "original":
        return image
    elif technique == "sharpen":
        return image.filter(ImageFilter.SHARPEN)
    elif technique == "contrast":
        enhancer = ImageEnhance.Contrast(image)
        return enhancer.enhance(2.0)
    elif technique == "blur":
        return image.filter(ImageFilter.GaussianBlur(radius=2))
    elif technique == "edge":
        return image.filter(ImageFilter.FIND_EDGES)

# Step 3: Create Custom Dataset with Processing
class ProcessedImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None, technique='original'):
        self.dataset = datasets.ImageFolder(root=root_dir)
        self.transform = transform
        self.technique = technique

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        path, label = self.dataset.samples[idx]
        image = Image.open(path).convert("RGB")
        image = apply_image_processing(image, self.technique)
        if self.transform:
            image = self.transform(image)
        return image, label

# Step 4: Transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Path to data (Update accordingly)
data_path = r"C:\Users\abhis\OneDrive\Desktop\image processing project\Plant Leaf Disease Detection Using Deep Learning - A Multi-Dataset Approach (Web sourced Dataset)"


# Step 5: Load Data
techniques = ['original', 'sharpen', 'contrast', 'blur', 'edge']
datasets_dict = {
    tech: ProcessedImageDataset(os.path.join(data_path, "train"), transform=transform, technique=tech)
    for tech in techniques
}
test_dataset = ProcessedImageDataset(os.path.join(data_path, "test"), transform=transform, technique='original')

# Step 6: Simple CNN Model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=16):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*32*32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# Step 7: Training & Evaluation
def train_and_evaluate(train_data, test_data, technique):
    print(f"\nTraining for technique: {technique}")
    model = SimpleCNN(num_classes=16).to(device)
    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=16)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):  # reduce for fast testing
        model.train()
        running_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Evaluate
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    print(f"Test Accuracy for {technique}: {acc:.2f}%")
    return acc

# Step 8: Run for All Techniques
results = {}
for tech in techniques:
    acc = train_and_evaluate(datasets_dict[tech], test_dataset, tech)
    results[tech] = acc

# Step 9: Visualize Results
plt.bar(results.keys(), results.values(), color='skyblue')
plt.ylabel("Accuracy (%)")
plt.title("Accuracy Comparison Across Image Processing Techniques")
plt.xticks(rotation=30)
plt.show()



Training for technique: original
